In [1]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver
from ttictoc import tic, toc

mesh = ngs.Mesh('whatever.vol')

# mesh.Refine()
mesh.Curve(3)

In [2]:
%run J.ipynb

using 3d mesh with ne= 63966  elements and nv= 11092  points
Solving...
Number of iterations = 2980/10000 | Residual = 9.98430314643603e-06
2999.9999999999886
Solving...
Number of iterations = 789/10000 | Residual = 9.83900277470855e-06
2999.999994803527


In [3]:
from bhdata import BHCurves
# fun_dw  = BHCurves(3)
# fun_w   = fun_dw.Integrate()
# fun_ddw = fun_dw.Differentiate()

from bhdata import Brauer, BrauerCut
fun_w, fun_dw, fun_ddw  = BrauerCut()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil|ambient|default"
nonlinear = "r_steel|l_steel|mid_steel"

lin = 0

In [4]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

HCurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'ambient_face')
# HCurl = ngs.HCurl(mesh, order = deg, nograds = True)
u,v = HCurl.TnT()


# Nonlinear:

maxit = 2000
tol2 = 1e-12
regb = 1e-8

A = ngs.GridFunction(HCurl)
B = ngs.curl(A)
normB = ngs.sqrt(B*B + regb)


if lin == 1: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: nu0/2*B*B}, default = nu0/2*B*B).Compile()
else: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B).Compile()

def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - ngs.curl(A)*Hs, mesh, order = 2*deg)
    # with ngs.TaskManager(): res = ngs.Integrate(cf_energy - ngs.curl(Hs)*A, mesh)
    return res

ir = ngs.IntegrationRule(ngs.fem.ET.TET, order = 2*deg)


if lin == 1: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: nu0}, default = nu0).Compile()
else: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

rhs = ngs.LinearForm(HCurl)
rhs += ngs.SymbolicLFI(cf_rhs*B*ngs.curl(v) - ngs.curl(v)*Hs, intrule = ir)

# rhs = ngs.LinearForm((cf_rhs*B*ngs.curl(v) - ngs.curl(v)*Hs)*ngs.dx)
# rhs = ngs.LinearForm((cf_rhs*B*ngs.curl(v) - ngs.curl(Hs)*v)*ngs.dx)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs


Id = ngs.CF((1,0,0,
             0,1,0,
             0,0,1), dims=(3,3))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1], B[0]*B[2],
              B[1]*B[0], B[1]*B[1], B[1]*B[2],
              B[2]*B[0], B[2]*B[1], B[2]*B[2]), dims = (3,3))


fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

if lin == 1: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: nu0*Id}, default = nu0*Id).Compile()
else: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id).Compile()

K_iter = ngs.BilinearForm(HCurl)
K_iter += ngs.SymbolicBFI(cf_iter*ngs.curl(u)*ngs.curl(v), intrule = ir)
# K_iter += (cf_iter*ngs.curl(u)*ngs.curl(v))*ngs.dx
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [5]:
print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,HCurl.ndof, "DOFs.\n ")

with ngs.TaskManager(): A.Set(ngs.CF((0,0,0)))

du = ngs.GridFunction(HCurl)
uo = ngs.GridFunction(HCurl)
wo = 1e12

for it in range(1,maxit+1):
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    # du.vec.data = da.mat.Inverse(HCurl.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    # iterativeSolver = CGSolver(K_iter.mat, freedofs = HCurl.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
    with ngs.TaskManager():
        iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-8,  maxiter = maxit)
        du.vec.data = iterativeSolver * dw.vec
        # du.vec.data = da.mat.Inverse(HCurl.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    
    if len(iterativeSolver.residuals) == maxit: print("... Failure!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm
    
    # wn = 1e12
    if abs(wo-w) < tol2:
    # if abs(wn-w) < tol2:
    # if nrm/nrm0 < tol2:
        print("converged to desired tolerance")
        break
    # elif abs(wo-w) < tol2*1e-2:
    #     print("stopped early due to stagnation")
    #     break
    else:
        # linesearch
        uo.vec.data = A.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            A.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.1*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                A.vec.data = uo.vec.data

Using 3D mesh with ne= 63966 elements and nv= 11092 points and  908625 DOFs.
 
Iter:  1 | assem : 6.15s | CG took 24.48s with 1118 iterations | alpha : 0.50 | energy = 0.0000000003 | relres = 1.00e+00 |
Iter:  2 | assem : 6.00s | CG took 22.27s with 1041 iterations | alpha : 1.00 | energy = -1.2615011758 | relres = 1.25e-01 |
Iter:  3 | assem : 5.92s | CG took 18.52s with  880 iterations | alpha : 1.00 | energy = -1.4302583202 | relres = 1.88e-02 |
Iter:  4 | assem : 5.93s | CG took 19.92s with  938 iterations | alpha : 1.00 | energy = -1.4640375023 | relres = 2.20e-03 |
Iter:  5 | assem : 6.04s | CG took 19.58s with  888 iterations | alpha : 1.00 | energy = -1.4684062257 | relres = 2.55e-04 |
Iter:  6 | assem : 6.20s | CG took 18.30s with  794 iterations | alpha : 1.00 | energy = -1.4689200862 | relres = 3.52e-05 |
Iter:  7 | assem : 6.46s | CG took 17.43s with  745 iterations | alpha : 1.00 | energy = -1.4689900606 | relres = 2.96e-06 |
Iter:  8 | assem : 6.73s | CG took 16.38s with 

In [10]:
scalars = mesh.MaterialCF({"coil" : 1, "mid_steel|l_steel|r_steel" : 2}, default = 0)
L2 = ngs.L2(mesh, order = 0)
scal = ngs.GridFunction(L2)
scal.Set(scalars)

normB = ngs.sqrt(B*B + 1e-8)
H = B*mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

vtk = ngs.VTKOutput(mesh, coefs = [B,H,ngs.curl(Hs),scal],names = ["B","H","J","scalars"], filename = "B3", order = 2)
vtk.Do()

'B3'

In [7]:
import pyvista as pv
pv.global_theme.cmap = 'jet'

pvmesh = pv.read("B3.vtu")
pvmesh.set_active_scalars("scalars")
pvmesh.set_active_vectors("B")
threshed = pvmesh.threshold([0.7,1.6])
# threshed.set_active_scalars("scalars")

p = pv.Plotter()
p.add_mesh(threshed, style = 'surface', color = "w", opacity = 0, label = None)
threshed = pvmesh.threshold([1.2,3.1])
p.add_mesh(threshed, scalars = "B")

arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01) #0.8
p.add_mesh(arrows, color = "orange")

pvmesh.set_active_vectors("J")
arrows2 = pvmesh.glyph(scale = "J", orient = True, tolerance = 0.001, factor = 1*1e-8)
p.add_mesh(arrows2, color = "black")

p.show(jupyter_backend = "html")
# p.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [8]:
print(ngs.Integrate(ngs.Norm(B),mesh))
print(fun_W())

0.0003612008775077082
-1.468995755971984


In [9]:
pvmesh

UnstructuredGrid (0x2074d919780)
  N Cells:    63966
  N Points:   639660
  X Bounds:   -2.000e-01, 2.000e-01
  Y Bounds:   -2.000e-01, 2.000e-01
  Z Bounds:   -1.000e-01, 1.000e-01
  N Arrays:   3